In [ ]:
import pandas as pd

# testデータの読み込み
test_file_path = "/content/drive/My Drive/apartment_price_2024_fall/test/test.csv"
df = pd.read_csv(test_file_path, index_col=0, low_memory=False, encoding="utf-8")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19390 entries, 1000101 to 47017289
Data columns (total 26 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   種類          19390 non-null  object 
 1   地域          0 non-null      float64
 2   市区町村コード     19390 non-null  int64  
 3   都道府県名       19390 non-null  object 
 4   市区町村名       19390 non-null  object 
 5   地区名         19390 non-null  object 
 6   最寄駅：名称      19370 non-null  object 
 7   最寄駅：距離（分）   19370 non-null  object 
 8   間取り         18184 non-null  object 
 9   面積（㎡）       19390 non-null  int64  
 10  土地の形状       0 non-null      float64
 11  間口          0 non-null      float64
 12  延床面積（㎡）     0 non-null      float64
 13  建築年         18854 non-null  object 
 14  建物の構造       18499 non-null  object 
 15  用途          15162 non-null  object 
 16  今後の利用目的     18595 non-null  object 
 17  前面道路：方位     0 non-null      float64
 18  前面道路：種類     0 non-null      float64
 19  前面道路：幅員（ｍ）  0 non-nul

In [ ]:
df.head()

,種類,地域,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,...,今後の利用目的,前面道路：方位,前面道路：種類,前面道路：幅員（ｍ）,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等
ID,,,,,,,,,,,,,,,,,,,,,
1000101,中古マンション等,NaN,1101,北海道,札幌市中央区,大通西,西１８丁目,1,２ＤＫ,55,...,住宅,NaN,NaN,NaN,商業,80.0,400.0,2024年第1四半期,未改装,NaN
1000102,中古マンション等,NaN,1101,北海道,札幌市中央区,大通西,西１８丁目,1,１ＬＤＫ,55,...,住宅,NaN,NaN,NaN,商業,80.0,400.0,2024年第1四半期,改装済み,NaN
1000103,中古マンション等,NaN,1101,北海道,札幌市中央区,大通西,西１８丁目,0,２ＤＫ,35,...,住宅,NaN,NaN,NaN,商業,80.0,400.0,2024年第1四半期,未改装,NaN
1000105,中古マンション等,NaN,1101,北海道,札幌市中央区,大通西,西１８丁目,5,２ＬＤＫ,50,...,事務所,NaN,NaN,NaN,商業,80.0,400.0,2024年第1四半期,未改装,NaN
1000106,中古マンション等,NaN,1101,北海道,札幌市中央区,大通西,西１８丁目,5,１Ｋ,30,...,事務所,NaN,NaN,NaN,商業,80.0,400.0,2024年第1四半期,未改装,NaN


In [ ]:
!pip install pyarrow


# testデータの前処理

In [ ]:
import pandas as pd
import numpy as np
import datetime
import os

def data_pre(df):
    # 非null列のリストを作成して、非null列のみ残す
    nonnull_list = [col for col in df.columns if df[col].count() > 0]
    df = df[nonnull_list]
    df = df.drop("種類", axis=1, errors="ignore")
    df = df.drop("市区町村名", axis=1, errors="ignore")

    # 文字列から数値（分）に変換するマッピング
    dis_mapping = {
        "30分～60分": 45,
        "1H～1H30": 75,
        "1H30～2H": 105,
        "2H～": 120,
    }

    # マッピングを適用し、変換できる値はfloatに変換
    df["最寄駅：距離（分）"] = pd.to_numeric(df["最寄駅：距離（分）"].replace(dis_mapping), errors='coerce')

    # "2,000㎡以上"を2000に置換し、数値部分を抽出
    df["面積（㎡）"] = df["面積（㎡）"].astype(str).replace("2,000㎡以上", "2000")
    df["面積（㎡）"] = df["面積（㎡）"].str.extract(r'(\d+\.?\d*)').astype(float)

    # "戦前"を1926年に置換し、建築年を数値に変換
    # df["建築年"] = df["建築年"].astype(str).replace("戦前", "1926")  # astype(str)を追加
    # df["建築年"] = df["建築年"].str.extract(r'(\d{4})').astype(float)  # 4桁の数値を抽出してfloat型に変換

   # 建築年の変換 with improved handling for different formats
    def convert_year(value):
        if isinstance(value, float) or pd.isna(value):
            return np.nan  # Handle missing values

         # More flexible regex for year extraction
        match = re.search(r"(\d{2,4})", str(value))  # Match 2-4 digits
        if match:
            year_str = match.group(1)

            # More flexible era matching using 'contains'
            if "平成" in str(value):
                year = 36 - float(year_str)
            elif "令和" in str(value):
                year = 4 - float(year_str)
            elif "昭和" in str(value):
                year = 96 - float(year_str)
            elif str(value) == "戦前":
                year = 76
            else:
                try:
                    year = float(year_str)
                except ValueError:
                    year = np.nan

            return year
        else:
            return np.nan

    df["建築年"] = df["建築年"].apply(convert_year)


    # 四半期を小数点に変換する辞書
    quarter_mapping = {
        "1": 0.25,
        "2": 0.50,
        "3": 0.75,
        "4": 0.99
    }

    # `取引時点`の年と四半期を正規表現で抽出
    extracted_dates = df["取引時点"].str.extract(r"(\d{4})年第(\d)四半期")
    extracted_dates.columns = ["年", "四半期"]

    # `年`と`四半期`を数値に変換
    extracted_dates["年"] = pd.to_numeric(extracted_dates["年"], errors="coerce")
    extracted_dates["四半期"] = extracted_dates["四半期"].map(quarter_mapping).astype(float)

    # `年`と`四半期`を加算して`取引時点`を数値として表現
    df["取引時点"] = extracted_dates["年"] + extracted_dates["四半期"]

    # カテゴリカル変数の設定
    categorical_features = ["市区町村コード", "都道府県名", "地区名", "最寄駅：名称",
                            "間取り", "建物の構造", "用途", "今後の利用目的",
                            "都市計画", "改装", "取引の事情等"]

    for col in categorical_features:
        df[col] = df[col].astype("category")

    return df, categorical_features

# testデータの読み込みと前処理
test_file_path = "/content/drive/My Drive/apartment_price_2024_fall/test/test.csv"
df_test = pd.read_csv(test_file_path, index_col=0, low_memory=False, encoding="utf-8")
df_processed, categorical_features = data_pre(df_test)

# インデックスからIDを列に戻す
df_processed = df_processed.reset_index()

# Parquet形式で前処理後のデータを保存
save_dir = '/content/drive/My Drive/apartment_price_2024_fall/test'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

parquet_path = os.path.join(save_dir, 'test_proceeded_1114_1.parquet')
df_processed.to_parquet(parquet_path, index=False)

print(f"データをParquet形式で保存しました: {parquet_path}")

df_processed.info()
df_processed


データをParquet形式で保存しました: /content/drive/My Drive/apartment_price_2024_fall/test/test_proceeded_1114_1.parquet
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19390 entries, 0 to 19389
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   ID         19390 non-null  int64   
 1   市区町村コード    19390 non-null  category
 2   都道府県名      19390 non-null  category
 3   地区名        19390 non-null  category
 4   最寄駅：名称     19370 non-null  category
 5   最寄駅：距離（分）  19370 non-null  float64 
 6   間取り        18184 non-null  category
 7   面積（㎡）      19390 non-null  float64 
 8   建築年        18854 non-null  float64 
 9   建物の構造      18499 non-null  category
 10  用途         15162 non-null  category
 11  今後の利用目的    18595 non-null  category
 12  都市計画       19124 non-null  category
 13  建ぺい率（％）    19042 non-null  float64 
 14  容積率（％）     19042 non-null  float64 
 15  取引時点       19390 non-null  float64 
 16  改装         16134 non-null  category
 17  

,ID,市区町村コード,都道府県名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,今後の利用目的,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等
0,1000101,1101,北海道,大通西,西１８丁目,1.0,２ＤＫ,55.0,2016.0,ＲＣ,NaN,住宅,商業,80.0,400.0,2024.25,未改装,NaN
1,1000102,1101,北海道,大通西,西１８丁目,1.0,１ＬＤＫ,55.0,1996.0,ＲＣ,NaN,住宅,商業,80.0,400.0,2024.25,改装済み,NaN
2,1000103,1101,北海道,大通西,西１８丁目,0.0,２ＤＫ,35.0,1977.0,ＳＲＣ,NaN,住宅,商業,80.0,400.0,2024.25,未改装,NaN
3,1000105,1101,北海道,大通西,西１８丁目,5.0,２ＬＤＫ,50.0,1984.0,ＳＲＣ,NaN,事務所,商業,80.0,400.0,2024.25,未改装,NaN
4,1000106,1101,北海道,大通西,西１８丁目,5.0,１Ｋ,30.0,1984.0,ＳＲＣ,NaN,事務所,商業,80.0,400.0,2024.25,未改装,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19385,47016501,47211,沖縄県,安慶田,NaN,NaN,３ＬＤＫ,70.0,1989.0,ＲＣ,住宅,住宅,１中住専,60.0,200.0,2023.99,NaN,NaN
19386,47016502,47211,沖縄県,安慶田,NaN,NaN,１ＬＤＫ,45.0,1989.0,ＲＣ,住宅,住宅,１中住専,60.0,200.0,2023.99,未改装,NaN
19387,47016792,47211,沖縄県,住吉,NaN,NaN,２ＤＫ,45.0,1991.0,ＲＣ,住宅,住宅,１中住専,60.0,200.0,2023.99,改装済み,NaN
19388,47016793,47211,沖縄県,住吉,NaN,NaN,３ＬＤＫ,70.0,1991.0,ＳＲＣ,住宅,住宅,１種住居,60.0,200.0,2023.99,未改装,NaN


テストデータの予測を行う

In [ ]:
import pandas as pd
import lightgbm as lgb

# **カテゴリカル特徴量の定義（トレーニングと同じ列を指定）**
categorical_features = ["市区町村コード", "都道府県名", "市区町村名", "地区名",
                        "最寄駅：名称", "間取り", "建物の構造", "用途",
                        "今後の利用目的", "都市計画", "改装", "取引の事情等"]

# **モデルのロード**
model = lgb.Booster(model_file='/content/drive/My Drive/apartment_price_2024_fall/models/lightgbm_model_1109_1.txt')

# **テストデータの読み込み**
df_loaded = pd.read_parquet("/content/drive/My Drive/apartment_price_2024_fall/test/test_proceeded_1109_1.parquet")

# **ID列の存在確認**
if 'ID' not in df_loaded.columns:
    raise ValueError("テストデータにID列が必要です。")

# **IDをインデックスに設定**
df_loaded = df_loaded.set_index('ID')

# **カテゴリカル変数を再設定（念のため）**
for col in categorical_features:
    if col in df_loaded.columns:
        df_loaded[col] = df_loaded[col].astype('category')

# **予測を実行**
predict = model.predict(df_loaded, num_iteration=model.best_iteration)

# **提出用のDataFrameを作成（IDと予測結果を含む）**
submission = pd.DataFrame({
    "ID": df_loaded.index,
    "取引価格（総額）_log": predict
})

# **予測値の統計を確認（事後チェック）**
print(f"予測値の平均: {predict.mean()}, 標準偏差: {predict.std()}")

# **提出用のCSVとして保存**
submission.to_csv("/content/drive/My Drive/apartment_price_2024_fall/test/submission_1109_lightgbm.csv", index=False)

print("予測結果をsubmission_1109_lightgbm.csvに保存しました。")


予測値の平均: 6.975355164141524, 標準偏差: 0.33528623758488024
予測結果をsubmission_1109_lightgbm.csvに保存しました。
